In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import tensorflow as tf

from scipy import stats
from sklearn.metrics import r2_score
from tqdm import tqdm
from tensorflow.keras.regularizers import l2
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm
import seaborn as sns

# Data
from seird.model import data_generator, version_data_model

# Model
from bayesflow.bayesflow_model import BayesFlow
from bayesflow.networks import HeteroskedasticModel, SequenceNet
from bayesflow.losses import heteroskedastic_loss, maximum_likelihood_loss
from bayesflow.trainer import train_step

# Misc
from utils.misc import true_vs_estimated

%load_ext autoreload
%autoreload 2

In [ ]:
# Network hyperparameters
inv_meta = {
    'n_units': [64, 64, 64],
    'activation': 'elu',
    'w_decay': 0.0,
    'initializer': 'glorot_uniform'
}
n_inv_blocks = 5

# Optional if using the predefined summary nets
summary_meta = {
    'lstm_units': [192, 192, 192],
    'activation': 'elu',
    'w_decay': 0.0,
    'initializer': 'glorot_uniform'
}


# Forward model hyperparameters
parameter_names = [r'$\beta$', r'$\sigma$', r'$\gamma$', r'$\mu_I$']
theta_dim = len(parameter_names)
n_test = 500


# Training and optimizer hyperparameters
ckpt_file = "bayesflow_parameter_estimation_v3_dropping_E_tseries"
batch_size = 64
epochs = 100
iterations_per_epoch = 1000
n_samples_posterior = 2000

starter_learning_rate = 0.001
global_step = tf.Variable(0, dtype=tf.int32)
decay_steps = 1000
decay_rate = .95
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(starter_learning_rate, decay_steps, decay_rate)
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [ ]:
%%time
data_test = data_generator(n_test)

# Preprocessing untrained data
X_test = np.array(data_test['X'])
params_test = np.array(data_test['params'])

In [ ]:
print(type(X_test))
print(X_test.shape)
print(type(params_test))
print(params_test.shape)

In [ ]:
# Sanity checks for numerical stability
assert(np.sum(X_test == np.inf) == 0)
assert(np.sum(X_test == -np.inf) == 0)
assert(np.sum(X_test == np.nan) == 0)

In [ ]:
print(X_test.shape)
print(params_test.shape)

In [ ]:
# Drop E compartment tseries
X_test = np.delete(X_test, 1, axis=2)

In [ ]:
print(X_test.shape)
print(params_test.shape)

In [ ]:
# Instantiate model
summary_net = SequenceNet()
model = BayesFlow(inv_meta, n_inv_blocks, theta_dim, summary_net=summary_net, permute=True)

In [ ]:
true_vs_estimated(model, X_test, params_test, n_samples_posterior, parameter_names, figsize=(12, 4))

In [ ]:
checkpoint = tf.train.Checkpoint(step=global_step, optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(checkpoint, './checkpoints/{}'.format(ckpt_file), max_to_keep=3)
checkpoint.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

In [ ]:
# %%time
for ep in range(1, epochs + 1):
    with tqdm(total=iterations_per_epoch, desc='Training epoch {}'.format(ep)) as p_bar:
        losses = train_step(model=model, 
                            optimizer=optimizer,
                            loss_fn=maximum_likelihood_loss, 
                            iterations=iterations_per_epoch,
                            batch_size=batch_size,
                            p_bar=p_bar,
                            global_step=global_step) 

        # Manage checkpoint
        manager.save()

In [ ]:
# Sample from trained model
params_samples = model.sample(X_test, n_samples_posterior, to_numpy=True)

# For each tseries compute mean of sampled posteriors
# For each tseries, n_samples_posterior set of parameters were samples
params_samples_mean = params_samples.mean(axis=0)

In [ ]:
# Sampling sanity checks